In [1]:
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt

In [2]:
from scipy.ndimage import gaussian_filter
from sklearn.manifold import Isomap
from sklearn.model_selection import train_test_split
from math import floor

from data_cube import DataCube
from similarity_network_fusion import SNF, cumulated_euc_ts

In [3]:
dc = DataCube(
    subjects="all",
    gestures=["3", "4", "5", "6"],
    channels=["2", "4", "6", "8"],
    data_grp="parsed")
dc.load_data()
dc.normalize_modalities()
dc.rms_smooth(100, 50)

dc.get_min_obs(smooth=True)

In [4]:
subj_lab = []
gest_lab = []
arrays = []

for s, gdict in dc.data_set_smooth.items():
    for g, a in gdict.items():
        subj_lab.append(s)
        gest_lab.append(int(g[0]))
        arrays.append(a[:dc.min_obs, 1:-1]) # truncate all modalities to minimum length

In [5]:
# set to array and base at shift gestures down by 3 (i.e. to 0,1,2,3 instead of 3,4,5,6) for keras
gest_lab = np.array(gest_lab) - 3

---

In [6]:
def cumulated_ts_2(a1, a2):
    """
    cumulated version of the time series w/ euclidean distance
    in which we take the sum values over time as time increases
    and then apply the chosen metric.
    i, j - arrays of data points
    """
    return la.norm(a1.sum(axis=0)-a2.sum(axis=0))

---

In [7]:
# generate raw signal SSMs for each gesture
raw_ssm_lst = [np.zeros(shape=(a.shape[0], a.shape[0])) for a in arrays]
for n, a in enumerate(arrays):
    for i in range(a.shape[0]):
        for j in range(a.shape[0]):
            raw_ssm_lst[n][i,j] = cumulated_ts_2(a[i,:],a[j,:])
                
# smooth SSM images
for r, s in enumerate(raw_ssm_lst):
    raw_ssm_lst[r] = gaussian_filter(s, sigma=1)

In [8]:
# generate ISOMAP SSMs for each gesture

# initialize embedding
iso = Isomap(n_neighbors=3, n_components=1)

iso_ssm_lst = [np.zeros(shape=(a.shape[0], a.shape[0])) for a in arrays]
for n, a in enumerate(arrays):
    embed = iso.fit_transform(a)
    for i in range(embed.size):
        for j in range(embed.size):
            iso_ssm_lst[n][i,j] = cumulated_ts_2(embed[i,:], embed[j,:])
                
# smooth SSM images
for r, s in enumerate(iso_ssm_lst):
    iso_ssm_lst[r] = gaussian_filter(s, sigma=1)

In [9]:
# generate SNF images for each gesture

print(len(arrays))

snf_lst = []
for n, a in enumerate(arrays):
    if n % 100 == 0: print(n)
    snf = SNF(a, k=0.2, metric=cumulated_euc_ts)
    # calculate graph weights to find knn
    snf.calc_weights()
    snf.normalize_weights()
    # generate and normalize knn graphs
    snf.calc_knn_weights()
    snf.normalize_knn_weights()
    # fuse graphs
    snf.network_fusion(eta=1, iters=20)
    # save template to dict
    snf_lst.append(snf.fused_similarity_template)

# smooth SNF images
for r, s in enumerate(snf_lst):
    snf_lst[r] = gaussian_filter(s, sigma=1)

576
0
100
200
300
400
500


---
### Pre-Process images

In [10]:
# normalize and standardize raw signal SSMs for each gesture
for r, s in enumerate(raw_ssm_lst):
    s = (s-s.mean()) / s.std()

In [11]:
# normalize and standardize ISOMAP SSMs for each gesture
for r, s in enumerate(iso_ssm_lst):
    s = (s-s.mean()) / s.std()

In [12]:
# normalize and standardize SNF images for each gesture
for r, s in enumerate(snf_lst):
    s = (s-s.mean()) / s.std()

---

In [13]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import regularizers

np.random.seed(1)

Using TensorFlow backend.
/home/svois/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/svois/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/svois/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/svois/.local/lib/python3.6/site-packages/tensorflow/python/fra

In [15]:
inp_shp = (dc.min_obs,dc.min_obs,1)

In [16]:
#Instantiate an empty model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=inp_shp, kernel_size=(11,11), strides=(1,1), padding="valid"))
model.add(Activation("relu"))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid"))

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding="valid"))
model.add(Activation("relu"))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid"))

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="valid"))
model.add(Activation("relu"))

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="valid"))
model.add(Activation("relu"))

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding="valid"))
model.add(Activation("relu"))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid"))

# Passing it to a Fully Connected layer
model.add(Flatten())
# 1st Fully Connected Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation("relu"))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))

# 2nd Fully Connected Layer
model.add(Dense(4096))
model.add(Activation("relu"))
# Add Dropout
model.add(Dropout(0.4))

# 3rd Fully Connected Layer
model.add(Dense(1000))
model.add(Activation("relu"))
# Add Dropout
model.add(Dropout(0.4))

# Output Layer
model.add(Dense(4))
model.add(Activation("softmax"))

model.summary()

ValueError: Negative dimension size caused by subtracting 11 from 6 for 'conv2d_2/convolution' (op: 'Conv2D') with input shapes: [?,6,6,96], [11,11,96,256].

In [ ]:
# Compile the model
opt = Adam(learning_rate=1e-5, beta_1=0.9, beta_2=0.999, amsgrad=False)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer="adam",
              metrics=["accuracy"])

In [ ]:
# train/ test
x_train, x_test, y_train, y_test = train_test_split(pad_ssm_lst, gest_lab, test_size=0.2, random_state=1)

# transform to numpy arrays
x_train = np.array(x_train).reshape(len(x_train), 63, 63, 1)
x_test = np.array(x_test).reshape(len(x_test), 63, 63, 1)
y_train = to_categorical(np.array(y_train))
y_test = to_categorical(np.array(y_test))

In [ ]:
batch_size = int(len(gest_lab) / 2)
epochs = 200

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    shuffle=True)

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

Flattened to 30% around 195 epochs; most recent run includes regularizers

In [ ]:
history.history["val_accuracy"]

---

In [ ]:
#Instantiate an empty model_iso
model_iso = Sequential()

# 1st Convolutional Layer
model_iso.add(Conv2D(filters=96, input_shape=inp_shp, kernel_size=(11,11), strides=(1,1), padding="valid"))
model_iso.add(Activation("relu"))
# Max Pooling
model_iso.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid"))

# 2nd Convolutional Layer
model_iso.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding="valid"))
model_iso.add(Activation("relu"))
# Max Pooling
model_iso.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid"))

# 3rd Convolutional Layer
model_iso.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="valid"))
model_iso.add(Activation("relu"))

# 4th Convolutional Layer
model_iso.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="valid"))
model_iso.add(Activation("relu"))

# 5th Convolutional Layer
model_iso.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding="valid"))
model_iso.add(Activation("relu"))
# Max Pooling
model_iso.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid"))

# Passing it to a Fully Connected layer
model_iso.add(Flatten())
# 1st Fully Connected Layer
model_iso.add(Dense(4096, input_shape=(224*224*3,)))
model_iso.add(Activation("relu"))
# Add Dropout to prevent overfitting
model_iso.add(Dropout(0.4))

# 2nd Fully Connected Layer
model_iso.add(Dense(4096))
model_iso.add(Activation("relu"))
# Add Dropout
model_iso.add(Dropout(0.4))

# 3rd Fully Connected Layer
model_iso.add(Dense(1000))
model_iso.add(Activation("relu"))
# Add Dropout
model_iso.add(Dropout(0.4))

# Output Layer
model_iso.add(Dense(4))
model_iso.add(Activation("softmax"))

model_iso.summary()

In [ ]:
# Compile the model
opt = Adam(learning_rate=1e-5, beta_1=0.9, beta_2=0.999, amsgrad=False)

model_iso.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer="adam",
                  metrics=["accuracy"])

In [ ]:
# train/ test
x_train, x_test, y_train, y_test = train_test_split(pad_iso_lst, gest_lab, test_size=0.2, random_state=1)

# transform to numpy arrays
x_train = np.array(x_train).reshape(len(x_train), 63, 63, 1)
x_test = np.array(x_test).reshape(len(x_test), 63, 63, 1)
y_train = to_categorical(np.array(y_train))
y_test = to_categorical(np.array(y_test))

In [ ]:
batch_size = int(len(gest_lab) / 2)
epochs = 200

history = model_iso.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        shuffle=True)

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
history.history["val_accuracy"]

---

In [ ]:
#Instantiate an empty model_snf
model_snf = Sequential()

# 1st Convolutional Layer
model_snf.add(Conv2D(filters=96, input_shape=inp_shp, kernel_size=(11,11), strides=(1,1), padding="valid"))
model_snf.add(Activation("relu"))
# Max Pooling
model_snf.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid"))

# 2nd Convolutional Layer
model_snf.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding="valid"))
model_snf.add(Activation("relu"))
# Max Pooling
model_snf.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid"))

# 3rd Convolutional Layer
model_snf.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="valid"))
model_snf.add(Activation("relu"))

# 4th Convolutional Layer
model_snf.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="valid"))
model_snf.add(Activation("relu"))

# 5th Convolutional Layer
model_snf.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding="valid"))
model_snf.add(Activation("relu"))
# Max Pooling
model_snf.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid"))

# Passing it to a Fully Connected layer
model_snf.add(Flatten())
# 1st Fully Connected Layer
model_snf.add(Dense(4096, input_shape=(224*224*3,)))
model_snf.add(Activation("relu"))
# Add Dropout to prevent overfitting
model_snf.add(Dropout(0.4))

# 2nd Fully Connected Layer
model_snf.add(Dense(4096))
model_snf.add(Activation("relu"))
# Add Dropout
model_snf.add(Dropout(0.4))

# 3rd Fully Connected Layer
model_snf.add(Dense(1000))
model_snf.add(Activation("relu"))
# Add Dropout
model_snf.add(Dropout(0.4))

# Output Layer
model_snf.add(Dense(4))
model_snf.add(Activation("softmax"))

model_snf.summary()

In [ ]:
# Compile the model
opt = Adam(learning_rate=1e-5, beta_1=0.9, beta_2=0.999, amsgrad=False)

model_snf.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer="adam",
                  metrics=["accuracy"])

In [ ]:
# train/ test
x_train, x_test, y_train, y_test = train_test_split(pad_snf_lst, gest_lab, test_size=0.2, random_state=1)

# transform to numpy arrays
x_train = np.array(x_train).reshape(len(x_train), 63, 63, 1)
x_test = np.array(x_test).reshape(len(x_test), 63, 63, 1)
y_train = to_categorical(np.array(y_train))
y_test = to_categorical(np.array(y_test))

In [ ]:
batch_size = int(len(gest_lab) / 2)
epochs = 200

history = model_snf.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        shuffle=True)

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

Got into 32% around 200 epochs; most recent run includes regularizers

In [ ]:
history.history["val_accuracy"]